# Login to Synapse
1. Sign up for the segmentation task [here](https://www.synapse.org/brats2021)
2. Add login details into kaggle secrets in Add-ons.

In [ ]:
!pip install synapseclient
import os
import time
import zipfile
import synapseclient
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("password")
secret_value_1 = user_secrets.get_secret("synapse_username")
syn = synapseclient.Synapse()
syn.login(secret_value_1, secret_value_0) 

# Download Training and Validation zip files from Synapse servers

In [ ]:
t = time.time()
train_obj = syn.get(entity='syn25956772')
test_obj = syn.get(entity='syn26017056')
print(f'Time elapsed in acquiring data from Synapse.org is', time.time()-t)
train_path = train_obj.path
test_path = test_obj.path
print(train_path)
print(test_path)

# Extract the zip files

In [ ]:
t = time.time()
with zipfile.ZipFile(train_path, 'r') as zip_ref:
    zip_ref.extractall('./')
with zipfile.ZipFile(test_path, 'r') as zip_ref:
    zip_ref.extractall('./')
print(f'Time elapsed in extracting the zip files is', time.time()-t)

os.rename('./RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021', './train')
os.rename('./RSNA_ASNR_MICCAI_BraTS2021_ValidationData', './test')

# Compare Patient IDs of task 1 and 2 data to find missing data in classification task

In [ ]:
segment_train_data_path = './train/'
segment_test_data_path = './test/'

for i, name in enumerate(os.listdir(segment_train_data_path)):
    dst = name.replace('BraTS2021_','')
    src = segment_train_data_path + name
    dst = segment_train_data_path + dst
    os.rename(src, dst)
    
for i, name in enumerate(os.listdir(segment_test_data_path)):
    dst = name.replace('BraTS2021_','')
    src = segment_test_data_path + name
    dst = segment_test_data_path + dst
    os.rename(src, dst)

segment_train_data = sorted(os.listdir(segment_train_data_path))
segment_test_data = sorted(os.listdir(segment_test_data_path))

class_train_data_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
class_test_data_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'

class_train_data = sorted(os.listdir(class_train_data_path))
class_test_data = sorted(os.listdir(class_test_data_path))


segment_train_set = set(segment_train_data)
segment_test_set = set(segment_test_data)

class_train_set = set(class_train_data)
class_test_set = set(class_test_data)

extra_in_seg_train = list(sorted(segment_train_set-class_train_set))
extra_in_seg_test = list(sorted(segment_test_set-class_test_set))

extra_in_class_train = list(sorted(class_train_set-segment_train_set))
extra_in_class_test = list(sorted(class_test_set-segment_test_set))

# No. of extra elements in respective datasets
print('No. of extra elements in seg_train dataset', len(extra_in_seg_train))
print('No. of extra elements in seg_test dataset', len(extra_in_seg_test))
print('No. of extra elements in class_train dataset', len(extra_in_class_train))
print('No. of extra elements in class_test dataset', len(extra_in_class_test))

# print(extra_in_seg_train)
# print(extra_in_seg_test)

# print(any(item in class_test_data for item in segment_train_data))